In [1]:
from bs4 import BeautifulSoup, NavigableString
import cohere

api_key = "CYHh49rgFYiXv8Gl8sZ45myPlsglfc650p79U89i"

co = cohere.Client(api_key)

def generate_xpath(element, path=""):
    """ Recursive function to generate the xpath of an element """
    if element.parent is None:
        return path
    else:
        siblings = [sib for sib in element.parent.children if sib.name == element.name]
        if len(siblings) > 1:
            count = siblings.index(element) + 1
            path = f"/{element.name}[{count}]{path}"
        else:
            path = f"/{element.name}{path}"
        return generate_xpath(element.parent, path)

def create_xpath_dict(html, only_body=True, max_length=200):
    soup = BeautifulSoup(html, 'html.parser')
    if only_body:
        root = soup.body
    else:
        root = soup.html
    element_xpath_list = []
    element_attributes_list = []
    stack = [(root, '')]  # stack to keep track of elements and their paths

    while stack:
        element, path = stack.pop()
        if element.name is not None:
            current_path = generate_xpath(element)
            element_attrs = dict(element.attrs)
            direct_text_content = ''.join([str(content).strip() for content in element.contents if isinstance(content, NavigableString) and content.strip()])
            if direct_text_content:
                element_attrs['text'] = direct_text_content
                element_attrs['element'] = element.name
                for key in element_attrs:
                    if len(element_attrs[key]) > max_length:
                        element_attrs[key] = element_attrs[key][:max_length]
                element_xpath_list.append(current_path)
                element_attributes_list.append(element_attrs)
            elif element_attrs != {}:
                element_attrs['element'] = element.name
                for key in element_attrs:
                    if len(element_attrs[key]) > max_length:
                        element_attrs[key] = element_attrs[key][:max_length]
                element_xpath_list.append(current_path)
                element_attributes_list.append(element_attrs)
            for child in element.children:
                if child.name is not None:
                    stack.append((child, current_path))

    return element_xpath_list, element_attributes_list

def get_results(query, html, top_n=5, model="rerank-english-v3.0", return_documents=True, rank_fields=['element', 'placeholder', 'text', 'name']):
    xpath_list, attributes_list = create_xpath_dict(html)
    assert len(xpath_list) == len(attributes_list)
    l = len(xpath_list)
    list_of_results = []
    for j in range(0, l, 1000):
        attr = attributes_list[j:j+1000]
        results = co.rerank(model=model, query=query, documents=attr, top_n=top_n, return_documents=return_documents, rank_fields=rank_fields)
        results = [r.dict() for r in results.results]
        for r in results:
            r['index']+=j
        list_of_results += results
    list_of_results = sorted(list_of_results, key=lambda x: x['relevance_score'], reverse=True)
    xpaths = [xpath_list[result['index']] for result in list_of_results[:top_n]]
    return xpaths, list_of_results[:top_n]

In [2]:
import pandas as pd
#df = pd.read_csv('gradio_dataset.csv')
df = pd.read_csv('gradio_dataset_correct.csv')
df.head()

,query,url,html,source_nodes,code,outer_html,code_run_success,correct_button,base_page_image,highlighted_page_image,model,html_id
0,click on create account,https://en.wikipedia.org/wiki/Main_Page,"<html class=""client-js vector-feature-language...",> Source (Doc id: 5c15d77f-5d73-4555-9253-72ca...,create_account_link = driver.find_element(By.X...,"<li id=""pt-createaccount-2"" class=""user-links-...",True,1.0,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,gpt-35-turbo,"<html backend_node_id=""1"" class=""client-js vec..."
1,click on past,https://news.ycombinator.com/,"<html lang=""en"" op=""news""><head><meta name=""re...",> Source (Doc id: a5545abb-f954-4073-aee6-ecdd...,link_to_click = driver.find_element(By.LINK_TE...,"<a href=""front"">past</a>",True,1.0,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,gpt-35-turbo,"<html backend_node_id=""1"" lang=""en"" op=""news"">..."
2,click on pricing,https://www.perfagents.com/,"<html lang=""en""><head>\n <meta charset=""utf...",> Source (Doc id: 87e5be48-b025-4fc2-a822-1ccb...,"pricing_link = driver.find_element(By.XPATH, ""...","<a class=""font-medium px-3 lg:px-5 py-2 items-...",True,1.0,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,gpt-35-turbo,"<html backend_node_id=""1"" lang=""en""><head back..."
3,click on SPF Justice,https://justice.belgium.be/fr,"<html lang=""fr"" dir=""ltr"" prefix=""content: htt...",> Source (Doc id: 581e75ba-40b2-45b3-96be-fd43...,spf_justice_link = driver.find_element(By.XPAT...,<span>SPF Justice</span>,True,1.0,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,gpt-35-turbo,"<html backend_node_id=""1"" class=""js"" dir=""ltr""..."
4,click on Search,https://www.cancer.gov/publications/dictionari...,"<html lang=""en"" dir=""ltr"" prefix=""og: https://...",> Source (Doc id: 70ae9e3b-f11c-46bc-bcac-d6a1...,"search_button = driver.find_element(By.XPATH, ...","<input type=""submit"" class=""submit button post...",True,1.0,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,iVBORw0KGgoAAAANSUhEUgAABkAAAAOECAYAAAD5Tf2iAA...,gpt-35-turbo,"<html backend_node_id=""1"" dir=""ltr"" lang=""en"" ..."


In [3]:
i=10
results = get_results(df.iloc[i]['query'], df.iloc[i]['html'])
print(df.iloc[i]['query'])

click on search bar Search destination


In [6]:
results

(['/html/body/div[5]/div/div/div[1]/div/div[3]/div[2]/div/div/div/header/div/div[2]/div[2]/div/div/div/form/div[2]/div/div[1]/div[1]/label/div/input',
  '/html/body/div[5]/div/div/div[1]/div/div[3]/div[2]/div/div/div/header/div/div[2]/div[2]/div/div/div/form/div[2]/div/div[5]/div[1]/div[2]/button/div/div[2]',
  '/html/body/div[5]/div/div/div[1]/div/div[3]/div[2]/div/div/div/header/div/div[3]/nav',
  '/html/body/noscript/div',
  '/html/body/link[5]'],
 [{'document': {'aria-autocomplete': 'none',
    'aria-describedby': 'bigsearch-query-location-description',
    'autocomplete': 'off',
    'autocorrect': 'off',
    'class': ['i18z192n',
     'atm_9s_1ulexfb',
     'atm_3f_idpfg4',
     'atm_gi_idpfg4',
     'atm_l8_idpfg4',
     'atm_vy_1osqo2v',
     'atm_26_glywfm',
     'atm_c8_km0zk7',
     'atm_g3_18khvle',
     'atm_fr_1m9t47k',
     'atm_cs_9dzvea',
     'atm_7l_dezgoh',
     'atm_sq_1l2sidv',
     'atm_c8_km0zk7_17x46du',
     'atm_g3_18khvle_17x46du',
     'atm_fr_1m9t47k_17x46d